# 14.2 词嵌入近似训练
- **目录**
  - 14.2.1 负采样
  - 14.2.2 层序Softmax


- 近似训练（Approximate Training）指的是一类通过**简化**或**逼近计算密集型步骤**来提升训练效率的技术，尤其在处理**长序列**、**大规模词汇表**或**复杂模型**时。
- 这些方法通过**牺牲**部分理论精确性来换取计算速度和内存效率的提升。
- 近似训练的核心思想：
  - **目标**：解决传统方法（如精确计算）在NLP任务中面临的**计算复杂度高**或**内存消耗大**的问题。
  - **手段**：用数学或工程上的**近似替代**精确计算，使模型在可接受的误差范围内高效训练。
- 近似训练技术一般包括：负采样（Negative Sampling）、层序Softmax（Hierarchical Softmax）、采样Softmax（Sampled Softmax）、掩码语言模型（Masked Language Model, MLM）的局部预测、低秩近似（Low-Rank Approximation）、梯度近似（Gradient Approximation）。

- 14.1节中的讨论：
  - 跳元模型的主要思想是使用softmax运算来计算基于给定的中心词$w_c$生成上下文词$w_o$的条件概率（如公式14.1.4），对应的对数损失在公式14.1.7给出。
  - 由于softmax操作的性质，上下文词可以是词表$\mathcal{V}$中的任意项，公式14.1.7包含与整个词表大小一样多的项的求和。
  - 因此， **公式14.1.8中跳元模型的梯度计算和公式14.1.15中的连续词袋模型的梯度计算都包含求和**。
  - 不幸的是，在**一个词典上（通常有几十万或数百万个单词）求和的梯度的计算成本是巨大的**！
- 为了降低上述计算复杂度，本节将介绍两种近似训练方法：**负采样**和**分层softmax**。
- 由于跳元模型和连续词袋模型的相似性，我们将以跳元模型为例来描述这两种近似训练方法。

## 14.2.1 负采样


负采样修改了原目标函数。给定中心词$w_c$的上下文窗口，任意上下文词$w_o$来自该上下文窗口的被认为是由下式建模概率的事件：

$$P(D=1\mid w_c, w_o) = \sigma(\mathbf{u}_o^\top \mathbf{v}_c),\tag{14.2.1}$$

其中$\sigma$使用了sigmoid激活函数的定义：

$$\sigma(x) = \frac{1}{1+\exp(-x)}.\tag{14.2.2}$$


让我们从最大化文本序列中所有这些事件的联合概率开始训练词嵌入。具体而言，给定长度为$T$的文本序列，以$w^{(t)}$表示时间步$t$的词，并使上下文窗口为$m$，考虑最大化联合概率：

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(D=1\mid w^{(t)}, w^{(t+j)}).\tag{14.2.3}$$


然而， 公式14.2.3只考虑那些**正样本的事件**。仅当所有词向量都等于无穷大时，公式14.2.3中的联合概率才最大化为1。当然，这样的结果毫无意义。为了使目标函数更有意义，**负采样**添加从预定义分布中采样的负样本。

用$S$表示上下文词$w_o$来自中心词$w_c$的上下文窗口的事件。对于这个涉及$w_o$的事件，从预定义分布$P(w)$中采样$K$个不是来自这个上下文窗口**噪声词**。用$N_k$表示噪声词$w_k$（$k=1, \ldots, K$）不是来自$w_c$的上下文窗口的事件。假设正例和负例$S, N_1, \ldots, N_K$的这些事件是相互独立的。负采样将公式14.2.3中的联合概率（仅涉及正例）重写为

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}),\tag{14.2.4} $$

通过事件$S, N_1, \ldots, N_K$近似条件概率：

$$ P(w^{(t+j)} \mid w^{(t)}) =P(D=1\mid w^{(t)}, w^{(t+j)})\prod_{k=1,\ w_k \sim P(w)}^K P(D=0\mid w^{(t)}, w_k).\tag{14.2.5}$$


分别用$i_t$和$h_k$表示词$w^{(t)}$和噪声词$w_k$在文本序列的时间步$t$处的索引。公式14.2.5中关于条件概率的对数损失为：

$$
\begin{aligned}
-\log P(w^{(t+j)} \mid w^{(t)})
=& -\log P(D=1\mid w^{(t)}, w^{(t+j)}) - \sum_{k=1,\ w_k \sim P(w)}^K \log P(D=0\mid w^{(t)}, w_k)\\
=&-  \log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \mathbf{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\left(1-\sigma\left(\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right)\right)\\
=&-  \log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \mathbf{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\sigma\left(-\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right).
\end{aligned} \tag{14.2.6}
$$

我们可以看到，现在每个训练步的梯度计算成本与词表大小无关，而是线性依赖于$K$。当将超参数$K$设置为较小的值时，在负采样的每个训练步处的梯度的计算成本较小。



- **要点：**
  - 负采样是一种用于训练词嵌入的技术，旨在解决标准的词嵌入训练在大词汇量上计算成本过高的问题。
  - 负采样通过引入负样本的采样来提高词嵌入模型的训练效率，使得模型的训练不再直接依赖于整个词表的大小，而是依赖于较小的负样本集合大小$K$，从而解决了大规模词汇集上的计算成本问题。
  - **目标概率模型**：给定中心词$w_c$和上下文词$w_o$，通过sigmoid函数定义的概率$P(D=1\mid w_c, w_o) = \sigma(\mathbf{u}_o^\top \mathbf{v}_c)$描述了$w_o$出现在$w_c$上下文中的概率。
  - **最大化联合概率**：训练目标是最大化文本序列中所有正样本（即实际上下文词对）事件的联合概率$\prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(D=1\mid w^{(t)}, w^{(t+j)})$。
  - **负采样方法**：为了使目标函数更实用，引入负采样技术，即从预定义分布$P(w)$中采样$K$个负样本（噪声词），假设所有正样本和负样本的事件是相互独立的。
  - **替代条件概率公式**：使用事件$S$（上下文词来自中心词的上下文窗口）和噪声词事件$N_k$，负采样将原有的条件概率公式14.2.3替换成$\prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)})$，通过这些事件近似。
  -  **对数损失函数**：条件概率的对数损失函数定义为$-\log P(w^{(t+j)} \mid w^{(t)}) = -  \log\, \sigma\left(\mathbf{u}_{i_{t+j}}^\top \mathbf{v}_{i_t}\right) - \sum_{k=1,\ w_k \sim P(w)}^K \log\sigma\left(-\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t}\right)$，涉及正样本和采样的负样本。
  -  **计算复杂性**：负采样方法将每步训练的梯度计算复杂性从依赖于词表大小降低到线性依赖于$K$。通过适当选择较小的$K$值，每步训练的计算成本大大降低。


-------------
- **说明：负采样的原理与步骤**
  - **负采样的基本原理：**
    - **问题背景**：在原始的跳元模型中，为了学习词向量，试图最大化正样本（即上下文窗口中的词对）的联合概率。为了达到这个目标，需要使用softmax来计算整个词汇表中每个词的概率，这在大型词汇表中是非常消耗计算资源的。
    - **目标函数的问题**：只考虑正样本的情况下，模型的目标函数会试图最大化所有正样本的联合概率。然而，如果只考虑正样本，这个目标函数的最优解是使所有词向量的值都变得非常大，这样的结果是没有意义的。
    - **负采样的引入**：为了解决上述问题，负采样被引入。除了考虑正样本，还随机选择一些负样本（或称为噪声词），这些词不在上下文窗口中。目标不再是最大化整个词汇表的联合概率，而是最大化正样本的概率，并最小化负样本的概率。
  - **负采样实现的步骤：**
    - **定义概率**：
      - 对于一个正样本（中心词 $w_c$ 和上下文词 $w_o$），定义这个词对出现的概率为：
    $$P(D=1\mid w_c, w_o) = \sigma(\mathbf{u}_o^\top \mathbf{v}_c).$$
        - 其中，$\sigma(x)$ 是sigmoid函数。
    - **定义目标函数**：
      - 最初，只考虑正样本，目标函数试图最大化所有正样本的联合概率：
      $$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(D=1\mid w^{(t)}, w^{(t+j)}).$$
      - 为了引入负样本，我们重新定义目标函数为：
      $$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}),$$
       - 其中条件概率是通过正样本和$K$个负样本来近似的：
      $$ P(w^{(t+j)} \mid w^{(t)}) =P(D=1\mid w^{(t)}, w^{(t+j)})\prod_{k=1,\ w_k \sim P(w)}^K P(D=0\mid w^{(t)}, w_k).$$
    - **计算损失函数**：
      - 使用上述定义的条件概率，我们可以定义损失函数为：
      $$-\log P(w^{(t+j)} \mid w^{(t)}).$$
      - 这样的损失函数旨在最大化正样本的概率并最小化负样本的概率。
    - **采样负样本**：
      - 从预定义的分布 $P(w)$ 中随机选择 $K$ 个噪声词作为负样本。
      - 这个分布通常根据单词频率的某种权重来选择，例如，使用单词频率的3/4次方作为权重。
    - **梯度下降**：
      - 使用上述损失函数进行梯度下降来更新词向量。
      - 由于只考虑了一个正样本和$K$个负样本，每次更新的计算成本与$K$线性相关，而与词汇表大小无关。
  - 负采样通过引入对抗性负样本，从根本上改变了优化问题的几何结构：
    - 打破对称性：防止所有向量坍缩到同一方向
    - 引入对比学习：迫使模型区分信号与噪声
    - 稳定数值优化：平衡梯度的**推拉**作用
    - 这种机制使得词向量能够收敛到具有**语义区分度**的稳定状态，而非趋向无穷大的病态解。
    - 实验表明，当负采样数$K=5-20$之间 时，即可获得高质量的语义表示。
---------------

## 14.2.2 层序Softmax

作为另一种近似训练方法，**层序Softmax（hierarchical softmax）** 使用二叉树（图14.2.1中说明的数据结构），其中树的每个叶节点表示词表$\mathcal{V}$中的一个词。
<center><img src='../img/hi-softmax.svg'></center>
<center>图14.2.1 用于近似训练的分层softmax，其中树的每个叶节点表示词表中的一个词</center><br>

用$L(w)$表示二叉树中表示字$w$的从根节点到叶节点的路径上的节点数（包括两端）。设$n(w,j)$为该路径上的$j^\mathrm{th}$节点，其上下文字向量为$\mathbf{u}_{n(w, j)}$。例如，图14.2.1中的$L(w_3) = 4$。层序softmax将公式14.1.4中的条件概率近似为

$$P(w_o \mid w_c) = \prod_{j=1}^{L(w_o)-1} \sigma\left( [\![  n(w_o, j+1) = \text{leftChild}(n(w_o, j)) ]\!] \cdot \mathbf{u}_{n(w_o, j)}^\top \mathbf{v}_c\right),\tag{14.2.7}$$

其中函数$\sigma$在公式14.2.2中定义，$\text{leftChild}(n)$是节点$n$的左子节点：如果$x$为真，$[\![x]\!] = 1$;否则$[\![x]\!] = -1$。


为了说明，让我们计算图14.2.1中给定词$w_c$生成词$w_3$的条件概率。这需要$w_c$的词向量$\mathbf{v}_c$和从根到$w_3$的路径（图14.2.1中加粗的路径）上的非叶节点向量之间的点积，该路径依次向左、向右和向左遍历：

$$P(w_3 \mid w_c) = \sigma(\mathbf{u}_{n(w_3, 1)}^\top \mathbf{v}_c) \cdot \sigma(-\mathbf{u}_{n(w_3, 2)}^\top \mathbf{v}_c) \cdot \sigma(\mathbf{u}_{n(w_3, 3)}^\top \mathbf{v}_c).\tag{14.2.8}$$

由$\sigma(x)+\sigma(-x) = 1$，它认为基于任意词$w_c$生成词表$\mathcal{V}$中所有词的条件概率总和为1：

$$\sum_{w \in \mathcal{V}} P(w \mid w_c) = 1.\tag{14.2.9}$$


幸运的是，由于二叉树结构，$L(w_o)-1$大约与$\mathcal{O}(\text{log}_2|\mathcal{V}|)$是一个数量级。当词表大小$\mathcal{V}$很大时，与没有近似训练的相比，使用分层softmax的每个训练步的计算代价显著降低。

-------------
- **说明：公式14.2.7，14.2.8，14.2.9的说明**
  - （1）**公式 14.2.7:** 此公式表示了给定中心词$w_c$下，生成外部词$w_o$的条件概率。  
    - $L(w_o)$：从二叉树的根到词$w_o$的路径上的节点数。
    - $n(w_o, j)$：从根到词$w_o$路径上的第j个节点。
    - $\mathbf{u}_{n(w_o, j)}$：第j个节点的向量表示。
    - $\mathbf{v}_c$：中心词$w_c$的向量表示。
    - $\sigma$：sigmoid函数，将输入值压缩到0到1之间。
    - 核心思想是，层序softmax不是直接计算词$w_o$的概率，而是按照从根到这个词的路径进行计算。
    - 对于路径上的每一个转向（向左或向右），都要计算一次概率。
  - （2）**公式 14.2.8:** 此公式是14.2.7公式的一个实例化。
    - 假设我们要计算生成词$w_3$的概率，该词在二叉树中的路径是左、右、左。因此需要计算三次概率。
  - （3）**公式 14.2.9:**
    - 该公式表示，给定中心词$w_c$，所有可能的外部词的条件概率之和为1。
    - 这符合概率的基本定义，即一个事件的所有可能结果的概率之和应该为1。
  - **举例:**
    - 假设词汇表中只有五个词: \{'a', 'b', 'c', 'd', 'e'\}。而我们的Huffman树结构如下（只为说明，并非真实的Huffman树）:
    ```
       ROOT
       /   \
      N1   'a'
     /  \
   'b'  N2
        /  \
      'c' 'd'
    ```
    - 'c'的路径是左、右、左，因此$P('c' \mid w_c)$ 为:
    $$\sigma(\mathbf{u}_{N1}^\top \mathbf{v}_c) \cdot \sigma(-\mathbf{u}_{N2}^\top \mathbf{v}_c)$$  
      其中，$\mathbf{u}_{N1}$和$\mathbf{u}_{N2}$是节点N1和N2的向量表示，$\mathbf{v}_c$是中心词$w_c$的向量表示。

- **公式14.2.7中$[\![ n(w_o, j+1) = \text{leftChild}(n(w_o, j)) ]\!]$的具体涵义**
  - 这个记号 $[\![ n(w_o, j+1) = \text{leftChild}(n(w_o, j)) ]\!]$是一个指示函数，代表了一个逻辑条件的真假值。
  - 公式里的$[\![ \cdot ]\!]$ 是一个**指示函数 (indicator function)** ：
    - 如果里面的条件为真，它返回 1。
    - 如果条件为假，它返回 -1。
  - 在这个特定的上下文中，条件$n(w_o, j+1) = \text{leftChild}(n(w_o, j))$ 检查路径上的下一个节点$n(w_o, j+1)$是否是当前节点$n(w_o, j)$的左子节点。
  - 因此：
    - 如果$n(w_o, j+1)$是$n(w_o, j)$的左子节点，这个函数返回 1。
    - 否则，它返回 -1（意味着它是右子节点）。
  - 这在层次Softmax中很有用，因为它帮助我们确定应该沿着二叉树的哪个方向（左或右）来计算特定词的概率。
  - 用一个简单的Python代码表示：
  ```python
    def indicator_function(n_wo_jp1, n_wo_j, leftChild):
        if n_wo_jp1 == leftChild(n_wo_j):
            return 1
        else:
            return -1
  ```
    - 在这里，`leftChild`是一个函数，给定一个节点，返回它的左子节点。

-----------------

## 小结

* 负采样通过考虑相互独立的事件来构造损失函数，这些事件同时涉及正例和负例。训练的计算量与每一步的噪声词数成线性关系。
* 分层softmax使用二叉树中从根节点到叶节点的路径构造损失函数。训练的计算成本取决于词表大小的对数。

-------------
- **附录：**
- **（1）负采样训练词嵌入示例**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import numpy as np
import random

# 示例文本数据
text = "I like natural language processing . I enjoy deep learning . I love machine learning ."
tokens = text.split()
vocab = set(tokens)
vocab_size = len(vocab)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# 计算词频
word_counts = Counter(tokens)
total_words = sum(word_counts.values())
word_freqs = {word: count / total_words for word, count in word_counts.items()}

# 负采样分布：P(w) = count(w)^(3/4) / sum(count(w)^(3/4))
pow_freqs = {word: count**0.75 for word, count in word_counts.items()}
sum_pow = sum(pow_freqs.values())
neg_sample_dist = {word: pow_freq / sum_pow for word, pow_freq in pow_freqs.items()}
neg_sample_words = list(neg_sample_dist.keys())
neg_sample_probs = list(neg_sample_dist.values())

# 超参数
embedding_dim = 10
window_size = 2
K = 5  # 每个正样本的负样本数
learning_rate = 0.01
epochs = 100

# 初始化词向量
center_embeddings = nn.Embedding(vocab_size, embedding_dim)  # v_c
context_embeddings = nn.Embedding(vocab_size, embedding_dim)  # u_o
nn.init.xavier_uniform_(center_embeddings.weight)
nn.init.xavier_uniform_(context_embeddings.weight)

# 优化器
optimizer = optim.SGD(list(center_embeddings.parameters()) + list(context_embeddings.parameters()), lr=learning_rate)

# 训练
for epoch in range(epochs):
    total_loss = 0
    for center_pos in range(len(tokens)):
        center_word = tokens[center_pos]
        center_idx = word_to_idx[center_word]
        
        # 获取上下文窗口
        context_window = range(max(0, center_pos - window_size), min(len(tokens), center_pos + window_size + 1))
        context_indices = [word_to_idx[tokens[pos]] for pos in context_window if pos != center_pos]
        
        for context_idx in context_indices:
            # 正样本损失: -log σ(u_o^T v_c)
            # 公式14.2.6的前半部分
            u_o = context_embeddings(torch.tensor(context_idx))
            v_c = center_embeddings(torch.tensor(center_idx))
            pos_score = torch.sigmoid(torch.dot(u_o, v_c))
            pos_loss = -torch.log(pos_score + 1e-10)  # 避免 log(0)
            
            # 负采样
            neg_samples = np.random.choice(neg_sample_words, size=K, p=neg_sample_probs, replace=True)
            neg_indices = [word_to_idx[word] for word in neg_samples]
            
            # 负样本损失: -sum log σ(-u_k^T v_c)
            # 公式14.2.6的后半部分
            neg_loss = 0
            for neg_idx in neg_indices:
                u_k = context_embeddings(torch.tensor(neg_idx))
                neg_score = torch.sigmoid(-torch.dot(u_k, v_c))
                neg_loss += -torch.log(neg_score + 1e-10)
            
            # 总损失
            loss = pos_loss + neg_loss
            total_loss += loss.item()
            
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss}")

# 获取训练后的词向量
trained_embeddings = center_embeddings.weight.data
for word, idx in word_to_idx.items():
    print(f"{word}: {trained_embeddings[idx]}")
    

Epoch 0, Loss: 243.70747685432434
Epoch 10, Loss: 206.1396586894989
Epoch 20, Loss: 167.03803253173828
Epoch 30, Loss: 156.81356525421143
Epoch 40, Loss: 151.44770860671997
Epoch 50, Loss: 149.59182167053223
Epoch 60, Loss: 146.5918196439743
Epoch 70, Loss: 141.97285223007202
Epoch 80, Loss: 142.299910902977
Epoch 90, Loss: 139.7790914773941
like: tensor([ 0.5667, -0.3364, -0.9349,  0.5797, -0.3512,  0.3652, -0.0976,  0.9614,
        -0.1616,  0.4742])
.: tensor([ 0.9286,  0.1652, -0.7736,  0.0022, -0.8984,  0.0215,  0.6049,  0.7349,
         0.0325,  0.1382])
I: tensor([-0.0333,  0.7369, -0.4710,  0.8120,  0.3424, -0.1277,  0.7740,  0.7851,
         0.3807, -0.0899])
language: tensor([ 0.3295,  0.4894,  0.0388,  0.9904,  0.9861,  0.5428, -0.0172,  1.3430,
         0.4378, -0.0017])
love: tensor([ 0.5455, -0.5593, -0.6580,  0.4329, -0.3569,  0.3043,  1.0169,  0.2203,
         1.0975,  0.6139])
processing: tensor([ 0.3267, -0.5965, -0.4262,  0.6374,  0.0799,  0.2888,  0.0458,  1.1187,
 

- **（2）层序Softmax训练词嵌入示例**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import numpy as np
import random
from collections import defaultdict, deque

# 示例文本数据
text = "I like natural language processing . I enjoy deep learning . I love machine learning ."
tokens = text.split()
vocab = list(set(tokens))
vocab_size = len(vocab)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for idx, word in enumerate(vocab)}

# 构建霍夫曼树（频率越高路径越短）
def build_huffman_tree(word_counts):
    """构建霍夫曼树，对应公式14.2.7中的二叉树结构"""
    # 创建叶节点
    nodes = [{'word': word, 'count': count, 'left': None, 'right': None, 'index': i} 
             for i, (word, count) in enumerate(word_counts.items())]
    node_count = len(nodes)
    
    # 构建优先队列（最小堆）
    heap = nodes.copy()
    heap.sort(key=lambda x: x['count'])
    
    # 构建内部节点
    while len(heap) > 1:
        # 取出两个最小频率的节点
        left = heap.pop(0)
        right = heap.pop(0)
        
        # 创建新内部节点
        new_node = {
            'word': None,
            'count': left['count'] + right['count'],
            'left': left,
            'right': right,
            'index': node_count
        }
        node_count += 1
        
        # 插入回堆中
        heap.append(new_node)
        heap.sort(key=lambda x: x['count'])
    
    return heap[0], node_count

# 计算词频
word_counts = defaultdict(int)
for word in tokens:
    word_counts[word] += 1

# 构建霍夫曼树
huffman_tree, total_nodes = build_huffman_tree(word_counts)

# 为每个词预计算路径和方向
word_paths = {}
word_codes = {}

def traverse(node, path=[], code=[]):
    """遍历霍夫曼树，记录每个词的路径和方向编码"""
    if node['word'] is not None:
        word_paths[node['word']] = path.copy()  # 存储路径节点索引
        word_codes[node['word']] = code.copy()  # 存储方向编码（1=左，0=右）
        return
    
    traverse(node['left'], path + [node['index']], code + [1])   # 左为1
    traverse(node['right'], path + [node['index']], code + [0])  # 右为0

traverse(huffman_tree)

# 超参数设置
embedding_dim = 10
window_size = 2
learning_rate = 0.01
epochs = 50

# 初始化词向量
center_embeddings = nn.Embedding(vocab_size, embedding_dim)  # 中心词向量v_c
node_embeddings = nn.Embedding(total_nodes, embedding_dim)   # 树节点向量u_n

# 初始化参数（Xavier初始化）
nn.init.xavier_uniform_(center_embeddings.weight)
nn.init.xavier_uniform_(node_embeddings.weight)

# 优化器
optimizer = optim.SGD(list(center_embeddings.parameters()) + list(node_embeddings.parameters()), lr=learning_rate)

# 训练循环
for epoch in range(epochs):
    total_loss = 0.0
    for center_pos in range(len(tokens)):
        center_word = tokens[center_pos]
        center_idx = torch.tensor([word_to_idx[center_word]])
        
        # 获取上下文窗口（公式14.2.7中的上下文词）
        context_indices = []
        start = max(0, center_pos - window_size)
        end = min(len(tokens), center_pos + window_size + 1)
        for pos in range(start, end):
            if pos != center_pos:
                context_word = tokens[pos]
                context_indices.append(word_to_idx[context_word])
        
        # 处理每个上下文词
        for context_idx in context_indices:
            context_word = idx_to_word[context_idx]
            
            # 获取该词的霍夫曼编码路径（对应公式14.2.7中的n(w,j)）
            path_indices = word_paths[context_word]  # 路径节点索引
            path_codes = word_codes[context_word]    # 路径方向编码
            
            # 初始化概率
            log_prob = 0.0
            
            # 获取中心词向量（公式14.2.7中的v_c）
            v_c = center_embeddings(center_idx)  # shape: (1, embedding_dim)
            
            # 沿着路径计算概率（公式14.2.7的乘积实现）
            for node_idx, code in zip(path_indices, path_codes):
                # 获取当前节点向量（公式14.2.7中的u_n(w,j)）
                u_n = node_embeddings(torch.tensor([node_idx]))  # shape: (1, embedding_dim)
                
                # 计算节点得分（公式14.2.7中的u_n(w,j)^T v_c）
                score = torch.dot(u_n.squeeze(), v_c.squeeze())
                
                # 根据方向计算概率（公式14.2.7中的[[...]]项）
                if code == 1:  # 左子节点
                    # σ(u_n^T v_c)
                    log_prob += torch.log(torch.sigmoid(score) + 1e-10)
                else:  # 右子节点
                    # σ(-u_n^T v_c) = 1 - σ(u_n^T v_c)
                    log_prob += torch.log(torch.sigmoid(-score) + 1e-10)
            
            # 损失是负对数概率（公式14.2.7取负对数）
            loss = -log_prob
            total_loss += loss.item()
            
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss}")  

# 验证：计算几个词的条件概率
def calculate_prob(center_word, target_word):
    """计算P(target_word|center_word)，对应公式14.2.7的实现"""
    if center_word not in word_to_idx or target_word not in word_to_idx:
        return 0.0
    
    center_idx = torch.tensor([word_to_idx[center_word]])
    v_c = center_embeddings(center_idx)
    
    path_indices = word_paths[target_word]  # 路径节点索引
    path_codes = word_codes[target_word]    # 路径方向编码
    
    prob = 1.0
    for node_idx, code in zip(path_indices, path_codes):
        u_n = node_embeddings(torch.tensor([node_idx]))
        score = torch.dot(u_n.squeeze(), v_c.squeeze())
        if code == 1:
            prob *= torch.sigmoid(score).item()  # 左子节点：σ(u_n^T v_c)
        else:
            prob *= torch.sigmoid(-score).item() # 右子节点：σ(-u_n^T v_c)
    return prob

# 示例验证（对应公式14.2.8的示例计算）
print("\n条件概率验证：")
print("P(learning|deep):", np.round(calculate_prob("deep", "learning"), 4))
print("P(learning|love):", np.round(calculate_prob("love", "learning"), 4))
print("P(deep|learning):", np.round(calculate_prob("learning", "deep"), 4))

# 打印最终词嵌入
print("\n最终词嵌入：")
for word, idx in word_to_idx.items():
    emb = center_embeddings.weight[idx].detach().numpy()
    print(f"'{word}': {np.round(emb, 4)}")

# 打印霍夫曼树路径示例
print("\n霍夫曼树路径示例：")
for word in ["deep", "learning", "machine"]:
    if word in word_paths:
        path = word_paths[word]
        codes = word_codes[word]
        print(f"'{word}': 路径节点={path}, 方向编码={codes}")

Epoch 0, Loss: 136.1333657503128
Epoch 10, Loss: 132.61492264270782
Epoch 20, Loss: 129.64000010490417
Epoch 30, Loss: 126.78521597385406
Epoch 40, Loss: 123.95440828800201

条件概率验证：
P(learning|deep): 0.1252
P(learning|love): 0.1218
P(deep|learning): 0.091

最终词嵌入：
'like': [-0.1786  0.6203  0.1859  0.5665 -0.1942 -0.2948 -0.4107  0.4355  0.5674
  0.1885]
'machine': [-0.3554 -0.2893  0.1604 -0.266  -0.2376  0.0983 -0.2284 -0.533   0.2093
  0.282 ]
'deep': [ 0.2617 -0.4536 -0.108  -0.6072 -0.2735 -0.0053 -0.573   0.2474 -0.4212
 -0.002 ]
'enjoy': [ 0.1784 -0.0188  0.1447  0.4478  0.2927 -0.3438 -0.2447 -0.1335 -0.0073
  0.3231]
'language': [-0.017   0.3465 -0.1796 -0.2872 -0.2294  0.0135  0.6566 -0.4332  0.8518
  0.1279]
'processing': [-0.5243  0.1001  0.1129  0.0735  0.0754  0.059  -0.1835  0.284   0.679
  0.2856]
'.': [-0.7736 -0.0764  1.0543 -0.6797  0.1605  0.4038  0.1273 -0.2054 -0.0908
  0.0843]
'love': [-0.1232  0.3277 -0.2041 -0.0485 -0.5394 -0.4037 -0.2045 -0.0509 -0.4663
  0.4051

---------